In [1]:
import pandas as pd

In [2]:
# Read the CSV file
df = pd.read_csv(r'U:\GitHub\research-pre-experiment\Mammo\outputs\VX vs Outcomes\All_XWalk_Outcome_cleaned.csv')

C:\Users\yuting.tseng\AppData\Local\Temp\ipykernel_18472\1285036749.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'U:\GitHub\research-pre-experiment\Mammo\outputs\VX vs Outcomes\All_XWalk_Outcome_cleaned.csv')


In [3]:
# Define the ethnicity mapping
ethnicity_mapping = {
    "AP": "Aboriginal People",
    "BL": "Black",
    "BR": "British Isles",
    "EA": "Asian - East and Southeast",
    "EE": "Eastern European",
    "FR": "French",
    "NE": "Northern European",
    "OT": "Other",
    "R": "Refused To Answer",
    "SA": "Asian - South",
    "SE": "South European",
    "U": "Unknown",
    "WE": "Western European"
}


In [4]:
# Map the short code to the descriptive label
df['ethnicity'] = df['ethnicity'].map(ethnicity_mapping)

# Convert 'screen_date' to datetime for proper sorting
df['screen_date'] = pd.to_datetime(df['screen_date'])

# 3. Sort by phn and screen_date
df = df.sort_values(by=['phn', 'screen_date'])

# 4. Assign a visit_id (1st visit = 1, 2nd visit = 2, etc.)
df['visit_id'] = df.groupby('phn').cumcount() + 1

In [5]:
# a) Total number of studies (rows) per ethnicity
total_studies = df.groupby('ethnicity').size().rename('total_studies')

# b) Number of unique patients per ethnicity
unique_patients = df.groupby('ethnicity')['phn'].nunique().rename('unique_patients')

# c) How many 1st, 2nd, 3rd, etc. visits per ethnicity
#    First, group by (ethnicity, visit_id) and count rows
visit_counts = df.groupby(['ethnicity', 'visit_id']).size().rename('count').reset_index()

# Convert long format to a pivot table
visits_pivot = visit_counts.pivot(index='ethnicity', columns='visit_id', values='count') \
                           .fillna(0).astype(int)

In [6]:
# Rename the columns for clarity: 1 -> one_time_studies, 2 -> two_time_studies, etc.
rename_dict = {}
for c in visits_pivot.columns:
    if c == 1:
        rename_dict[c] = 'one_time_studies'
    elif c == 2:
        rename_dict[c] = 'two_time_studies'
    elif c == 3:
        rename_dict[c] = 'three_time_studies'
    else:
        rename_dict[c] = f'{c}_time_studies'
visits_pivot = visits_pivot.rename(columns=rename_dict)

# Combine everything into a final summary DataFrame
summary = pd.concat([total_studies, unique_patients], axis=1)
summary = summary.join(visits_pivot, on='ethnicity').fillna(0).reset_index()

# Show the summary
print(summary)

                     ethnicity  total_studies  unique_patients  \
0            Aboriginal People           3336             2003   
1   Asian - East and Southeast          92111            34760   
2                Asian - South          15008             6178   
3                        Black           1264              527   
4                British Isles          85902            37744   
5             Eastern European          21439             9354   
6                       French           8842             3982   
7            Northern European           8013             3682   
8                        Other          20883             8719   
9            Refused To Answer           1355              569   
10              South European          12395             5062   
11            Western European          16578             7592   

    one_time_studies  two_time_studies  three_time_studies  4_time_studies  \
0               2003               888                 288     

In [7]:
# Group by ethnicity and phn to count the total number of visits per patient
patient_visit_counts = df.groupby(['ethnicity', 'phn']).size().reset_index(name='study_count')

# Create a new column for study category (e.g., "1 study", "2 studies", ..., "5+ studies")
def categorize_study_count(count):
    if count >= 5:
        return '5+ studies'
    else:
        return f'{count} studies'

patient_visit_counts['study_category'] = patient_visit_counts['study_count'].apply(categorize_study_count)

# Count the number of patients in each category per ethnicity
study_category_summary = patient_visit_counts.groupby(['ethnicity', 'study_category']) \
    .size().rename('patient_count').reset_index()

# Pivot the table for better readability
study_category_pivot = study_category_summary.pivot(index='ethnicity', columns='study_category', values='patient_count') \
    .fillna(0).astype(int).reset_index()

# Display the final summary
print(study_category_pivot)

study_category                   ethnicity  1 studies  2 studies  3 studies  \
0                        Aboriginal People       1115        600        199   
1               Asian - East and Southeast       8346       8548      10258   
2                            Asian - South       1713       1690       1746   
3                                    Black        147        154        138   
4                            British Isles      12302      12011       7912   
5                         Eastern European       2880       3054       2093   
6                                   French       1332       1336        762   
7                        Northern European       1270       1258        711   
8                                    Other       2495       2585       2213   
9                        Refused To Answer        170        170        128   
10                          South European       1322       1523       1374   
11                        Western European       257